In [18]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_auc_score,classification_report,recall_score,precision_score
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import numpy as np

In [19]:
import json
with open(r'./Folds/five_fold_cv.json', 'r') as file:
    file = json.load(file)


In [20]:
def get_results(acc_list,auc_list,recall_list,precision_list):    
    acc_list = np.array(acc_list)
    auc_list = np.array(auc_list)
    recall_list = np.array(recall_list)
    precision_list = np.array(precision_list)
    print(f'Recall, Average:{recall_list.mean():.4f}, Std:{recall_list.std():.4f}')
    print(f'precision, Average:{precision_list.mean():.4f}, Std:{precision_list.std():.4f}')
    print(f'Accuracy, Average:{acc_list.mean():.4f}, Std:{acc_list.std():.4f}')
    print(f'AUC, Average:{auc_list.mean():.4f}, Std:{auc_list.std():.4f}')

In [21]:
val_acc_list = []
val_auc_list = []
val_recall_list = []
val_precision_list = []

test_acc_list = []
test_auc_list = []
test_recall_list = []
test_precision_list = []
for fold in range(5):
    print(f"Taining Fold{fold+1} .....")
    test_file = file['test_split']
    train_file = file['cross_validation'][fold]['train_files']
    val_file = file['cross_validation'][fold]['validation_files']
    
    data = pd.read_csv(r'./data/Lung_data.csv')
    tarin_data = data[data['Image'].isin(train_file)]
    val_data = data[data['Image'].isin(val_file)]
    test_data = data[data['Image'].isin(test_file)]

    train_x = tarin_data.iloc[:,:-2]
    train_y = tarin_data[["label"]]

    val_x = val_data.iloc[:,:-2]
    val_y = val_data[["label"]]

    test_x = test_data.iloc[:,:-2]
    test_y = test_data[["label"]]
    
    standard_scaler = StandardScaler()
    train_x = standard_scaler.fit_transform(train_x)
    val_x = standard_scaler.transform(val_x)
    test_x = standard_scaler.transform(test_x)

    model = LogisticRegression()
    # Train the model with validation data
    model.fit(train_x, train_y)
    
    
    # -------------------validation----------------
    
    y_pred_proba = model.predict_proba(val_x)[:, 1]  # Get probabilities for the positive class
    y_pred = model.predict(val_x)

    accuracy = accuracy_score(val_y,  y_pred)
    # print(f'Accuracy: {accuracy:.4f}')
    val_acc_list.append(accuracy)
    
    auc_score = roc_auc_score(val_y, y_pred_proba)
    # print(f'AUC Score: {auc_score:.4f}')
    val_auc_list.append(auc_score)
    
    # Print the classification report
    recall = recall_score(val_y,  y_pred)
    # print(f'Recall Score: {recall:.4f}')
    val_recall_list.append(recall)
    
    precision = precision_score(val_y,  y_pred)
    # print(f'AUC Score: {precision:.4f}')
    val_precision_list.append(precision)
    
    
    # Get predicted probabilities
    # -------------------testing--------------
    
    y_pred_proba = model.predict_proba(test_x)[:, 1]  # Get probabilities for the positive class
    y_pred = model.predict(test_x)

    accuracy = accuracy_score(test_y,  y_pred)
    # print(f'Accuracy: {accuracy:.4f}')
    test_acc_list.append(accuracy)
    
    auc_score = roc_auc_score(test_y, y_pred_proba)
    # print(f'AUC Score: {auc_score:.4f}')
    test_auc_list.append(auc_score)
    

    recall = recall_score(test_y,  y_pred)
    # print(f'Recall Score: {recall:.4f}')
    test_recall_list.append(recall)
    
    precision = precision_score(test_y,  y_pred)
    # print(f'AUC Score: {precision:.4f}')
    test_precision_list.append(precision)

Taining Fold1 .....
Taining Fold2 .....
Taining Fold3 .....
Taining Fold4 .....


/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was 

Taining Fold5 .....


/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
print('Validation set')
get_results(val_acc_list,val_auc_list,val_recall_list,val_precision_list)
print('--------------------')
print("Test set")
get_results(test_acc_list,test_auc_list,test_recall_list,test_precision_list)
print('================================================')

Validation set
Recall, Average:0.6418, Std:0.0489
precision, Average:0.6879, Std:0.1047
Accuracy, Average:0.6936, Std:0.0548
AUC, Average:0.7237, Std:0.0477
--------------------
Test set
Recall, Average:0.7167, Std:0.0850
precision, Average:0.6972, Std:0.0592
Accuracy, Average:0.7882, Std:0.0343
AUC, Average:0.8530, Std:0.0442
